In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
N_CLASSES = 10

In [4]:
#Read in data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [27]:
#Parameters for a model
LEARNING_RATE = 0.001
BATCH_SIZE = 128
SKIP_STEP = 100
DROP_OUT = 0.75
N_EPOCHS = 10

In [6]:
#Create a placeholder for features,labels and dropout
with tf.name_scope("data"):
    X = tf.placeholder(tf.float32,[None,784],name = "X_placeholder")
    Y = tf.placeholder(tf.float32,[None,10], name = "Y_placeholder")
dropout = tf.placeholder(tf.float32, name = 'dropout')

In [16]:
#Create inference and weights
#model is conv - relu - pool - conv - relu - pool - fullly connected - softmax

global_step = tf.Variable(0, dtype= tf.int32, trainable = False, name = "global_step")

with tf.variable_scope('conv1',reuse = True) as scope:
    images = tf.reshape(X, shape = [-1,28,28,1])
    kernel = tf.get_variable('kernel',[5,5,1,32], initializer = tf.truncated_normal_initializer())
    biases = tf.get_variable('biases',[32],initializer = tf.random_normal_initializer())
    conv = tf.nn.conv2d(images,kernel, strides = [1,1,1,1], padding ="SAME")
    conv1 = tf.nn.relu(conv+biases, name = scope.name)
    # output is of dimension BATCH_SIZE x 28 x 28 x 32
    
with tf.variable_scope('pool1') as scope:
    pool1 = tf.nn.max_pool(conv1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
    # output is of dimension BATCH_SIZE x 14 x 14 x 32

with tf.variable_scope("conv2",reuse = True) as scope:
    kernel = tf.get_variable('kernel', [5,5,32,64], initializer = tf.truncated_normal_initializer())
    biases = tf.get_variable('biases',[64], initializer = tf.random_normal_initializer())
    conv = tf.nn.conv2d(pool1, kernel, strides=[1,1,1,1], padding = 'SAME')
    conv2 = tf.nn.relu(conv+biases, name = scope.name)
    # output is of dimension BATCH_SIZE x 14 x 14 x 64

with tf.variable_scope('pool2') as scope:
    pool2 = tf.nn.max_pool(conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
    # output is of dimension BATCH_SIZE x 7 x 7 x 64
    
with tf.variable_scope('fc',reuse = True) as scope:
    #use weight of dimension 7*7*64 X 1024
    input_features = 7*7*64
    w = tf.get_variable("weights", [input_features,1024], initializer = tf.truncated_normal_initializer())
    b = tf.get_variable('biases',[1024],initializer = tf.random_normal_initializer())
    #reshape pool2 to 2 dimensional
    pool2 = tf.reshape(pool2,[-1,input_features])
    fc = tf.nn.relu(tf.matmul(pool2,w) + b, name = 'relu')
    fc = tf.nn.dropout(fc, dropout,name = 'relu_dropout')

with tf.variable_scope('softmax_linear',reuse = None) as scope:
    w = tf.get_variable("weights",[1024,N_CLASSES],initializer = tf.truncated_normal_initializer())
    b = tf.get_variable('biases', [N_CLASSES], initializer = tf.random_normal_initializer())
    logits = tf.matmul(fc,w) + b

with tf.name_scope("loss"):
    entropy = tf.nn.softmax_cross_entropy_with_logits(logits =logits,labels =Y)
    loss = tf.reduce_mean(entropy, name = 'loss')
    

In [17]:
#Training Operation
optimizer = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss,global_step = global_step)

In [28]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    initial_step = global_step.eval()
    start_time = time.time()
    n_batches = int(mnist.train.num_examples/BATCH_SIZE)
    total_loss = 0.0
    for index in range(initial_step, n_batches * N_EPOCHS): # train the model n_epochs times
        X_batch, Y_batch = mnist.train.next_batch(BATCH_SIZE)
        _, loss_batch = sess.run([optimizer, loss], 
                                feed_dict={X: X_batch, Y:Y_batch, dropout: DROP_OUT}) 
        total_loss += loss_batch
        if (index + 1) % SKIP_STEP == 0:
            print('Average loss at step {}: {:5.1f}'.format(index + 1, total_loss / SKIP_STEP))
            total_loss = 0.0
    
    print("Optimization Finished!")
    print("Total time: {0} seconds".format(time.time() - start_time))
    
    # test the model
    n_batches = int(mnist.test.num_examples/BATCH_SIZE)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(BATCH_SIZE)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, logits], 
                                        feed_dict={X: X_batch, Y:Y_batch, dropout: DROP_OUT}) 
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
        total_correct_preds += sess.run(accuracy)   

    print("Accuracy {0}".format(total_correct_preds/mnist.test.num_examples))
    

Average loss at step 100: 7925.4
Average loss at step 200: 1509.2
Average loss at step 300: 973.7
Average loss at step 400: 654.6
Average loss at step 500: 487.8
Average loss at step 600: 409.5
Average loss at step 700: 352.0
Average loss at step 800: 281.1
Average loss at step 900: 242.7
Average loss at step 1000: 183.8
Average loss at step 1100: 186.4
Average loss at step 1200: 168.0
Average loss at step 1300: 167.5
Average loss at step 1400: 125.0
Average loss at step 1500: 131.2
Average loss at step 1600: 106.4
Average loss at step 1700: 114.0
Average loss at step 1800:  91.0
Average loss at step 1900:  87.3
Average loss at step 2000:  87.3
Average loss at step 2100:  77.7
Average loss at step 2200:  65.8
Average loss at step 2300:  64.2
Average loss at step 2400:  60.5
Average loss at step 2500:  54.6
Average loss at step 2600:  58.7
Average loss at step 2700:  43.7
Average loss at step 2800:  49.3
Average loss at step 2900:  50.0
Average loss at step 3000:  42.5
Average loss at s